# BAMBOO ANALYTICS 

> SQL based exploratory analysis for an investment based Fintech that helps customers buy stock locally and internationally. 

In [1]:
#Import useful libraries 

import pandas as pd
import numpy as np
import sqlite3, csv

In [2]:
#create a database if it does not exist 

con = sqlite3.connect('newdatabase.db')

In [3]:
#Connect

cursor = con.cursor()

In [4]:
#Create a table 

cursor.execute("CREATE TABLE deposites(id integer,amount integer,status text,user_id integer,payment_method_id integer,\
               inserted_at text,updated_at text,amount_paid real,fee real,exchange_rate text,deposit_type text,\
               dollar_instant_deposit_fee integer,dollar_processing_fee integer,currency text)")

In [5]:
#Read csv file dropping the headers

df = pd.read_csv('/content/deposits.csv', skiprows=[0])
df.to_csv('new', index=False)

In [6]:
#Load dataset into the table in the database

with open('/content/new', 'r') as file:
  records = csv.reader(file)
  no_records = 0
  for row in records:
    cursor.execute("INSERT INTO deposites VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)", row)
    con.commit()
    no_records += 1

In [7]:
#Get number of records in the new table deposites

select_all = "SELECT COUNT(*) FROM deposites"
rows = cursor.execute(select_all).fetchall()
 


print(rows)

[(1000,)]


In [8]:
#Get first 3 records in the table

select_a = "SELECT * FROM deposites LIMIT 3"
rows = cursor.execute(select_a).fetchall()
 


print(rows)

[(255917, 20, 'Deleted', 170, 7, '2021-05-11T00:18:19.000Z', '2021-05-11T00:19:25.000Z', 10070.0, 0.31, '$1/₦496.0', 'standard', 0, '0.31.1', 'NGN'), (255918, 20, 'Deleted', 155744, 2, '2021-05-11T00:42:27.000Z', '2021-05-11T00:44:00.000Z', 10058.88, 0.28, '$1/₦496.0', 'standard', 0, 0.28, 'NGN'), (255919, 300, 'Deleted', 161117, 7, '2021-05-11T00:44:18.000Z', '2021-05-11T00:44:25.000Z', 148950.0, 0.31, '$1/₦496.0', 'standard', 0, 0.31, 'NGN')]


# CREATE TABLE 2 "trades" in newdatabase

In [9]:
df = pd.read_csv('/content/trades.csv', skiprows=[0])

df.to_csv('data2', index=False)

In [10]:
cursor.execute("CREATE TABLE trades(id integer, price_per_share real,side text,stock_symbol text,\
               transaction_value  real,type text,user_id integer,quantity real,inserted_at text,\
               updated_at text,naira_fee real,dollar_fee real ,status text)")

In [11]:
with open('/content/data2', 'r') as file:
  records = csv.reader(file)
  no_records = 0
  for row in records:
    cursor.execute("INSERT INTO trades VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)", row)
    con.commit()
    no_records += 1

In [12]:
#Get number of records in the new table trades

select_all = "SELECT COUNT(*) FROM trades"
rows = cursor.execute(select_all).fetchall()
 


print(rows)

[(1000,)]


## SOLVE BUSINESS QUESTIONS USING QUERIES 
> Deposites

QUESTION 1: What is the median/average deposit value of those using payment method 3 in 2020?

In [13]:

qst1 = "SELECT AVG(amount_paid) AS average_deposit \
        FROM deposites \
        WHERE payment_method_id = 3 \
              AND inserted_at LIKE '2020%'"
qst1_answer = cursor.execute(qst1).fetchall()

print(qst1_answer)

[(256528.35,)]


QUESTION 2: When, in terms of which month of which year, did we observe the peak and valley (max and min) of deposit volume for Bamboo since inception?

In [14]:
qst2 = "SELECT strftime('%m-%Y', inserted_at) AS month_year, max(amount_paid), min(amount_paid) \
        FROM deposites"
qst2_answer = cursor.execute(qst2).fetchall()

print(qst2_answer)

[('05-2021', 248000000000.0, 20.58)]


QUESTION 3: Who are our power (most frequent) depositors by volume in 2021? Retrieve the user_id and the total volume of those users

In [15]:
qst3 = "SELECT DISTINCT user_id, COUNT(user_id) AS frequency \
        FROM deposites WHERE inserted_at LIKE '2021%' \
        GROUP BY user_id \
        ORDER BY frequency DESC \
        LIMIT 5"
qst3_answer = cursor.execute(qst3).fetchall()

print(qst3_answer)

[(53920, 21), (97170, 13), (154445, 8), (163896, 8), (163162, 7)]


QUESTION 4: We define High Net Worth individuals as people depositing $10k or above. Retrieve the user_id and the total deposits for users in that segment

In [16]:
qst4 = "SELECT user_id, SUM(amount_paid) AS deposit  \
        FROM deposites WHERE amount_paid > 10000 \
        GROUP BY user_id \
        ORDER BY deposit DESC \
        LIMIT 10"
qst4_answer = cursor.execute(qst4).fetchall()

print(qst4_answer)

[(162896, 248248002000.0), (166399, 27381500.0), (42131, 19865596.0), (53920, 19369597.279999997), (33074, 15621103.26), (166239, 11912150.0), (149505, 9945100.0), (122297, 9933920.0), (166171, 9932058.88), (163651, 9924150.0)]


QUESTION 5: What's the most used payment method among all users by volume?

In [17]:
qst5 = "SELECT  payment_method_id, COUNT ( payment_method_id) AS counts \
        FROM deposites \
        GROUP BY payment_method_id \
        ORDER BY counts DESC"
qst5_answer = cursor.execute(qst5).fetchall()

print(qst5_answer)

[(1, 384), (7, 346), (4, 135), (2, 84), (6, 31), (3, 20)]


QUESTION 6: What's the average transaction fee?

In [18]:
qst6 = "SELECT  AVG(fee) AS avg_transaction_fee \
       FROM deposites"
qst6_answer = cursor.execute(qst6).fetchall()

print(qst6_answer)

[(4.126549999999995,)]


## SECTION 2
> Trades

QUESTION 7: Obtain the month-over-month change in absolute value and percentage for all completed (where status=Filled)Trades in 2020?

In [19]:
#qst7 = "SELECT  AVG(fee) AS avg_transaction_fee FROM deposites"
#qst7_answer = cursor.execute(qst7).fetchall()

#print(qst7_answer)

QUESTION 8: Retrieve the list of user_id for those traded above 300x?

In [20]:
qst8 = "SELECT user_id, ROUND( SUM (price_per_share * quantity ), 2) AS traded_value \
       FROM trades \
       GROUP BY user_id \
       HAVING traded_value > 300 \
       ORDER BY traded_value DESC"
qst8_answer = cursor.execute(qst8).fetchall()

print(qst8_answer)

[(12651, 150000.0), (5805, 60000.0), (21530, 57382.62), (26287, 21350.0), (21939, 17236.0), (1635, 15156.0), (39799, 12764.84), (46739, 10274.0), (34209, 7880.0), (17930, 7373.16), (29944, 5679.0), (28918, 4952.22), (8479, 4900.0), (72395, 3874.56), (59124, 3820.41), (90381, 3455.01), (27089, 3252.95), (81501, 3083.13), (75507, 2908.89), (72699, 2871.83), (95020, 2636.24), (55387, 2632.0), (8179, 2352.1), (60514, 2350.0), (63007, 2205.5), (81127, 2082.38), (20228, 2001.87), (64293, 1711.87), (19217, 1644.0), (84831, 1553.81), (5140, 1540.1), (65758, 1515.22), (11958, 1485.0), (24819, 1481.66), (82708, 1477.5), (16811, 1475.0), (96689, 1455.34), (27389, 1435.77), (74530, 1410.0), (16457, 1337.44), (14837, 1241.96), (98354, 1201.7), (68131, 1174.83), (27481, 1164.37), (80574, 1132.07), (40879, 1112.0), (18390, 1090.51), (47067, 1079.13), (1262, 1070.51), (5095, 1049.29), (35415, 1034.84), (29459, 1023.79), (29491, 991.42), (721, 990.0), (75238, 989.71), (38525, 989.55), (65088, 985.08), 

QUESTION 9: Find the best performing week, by total transaction value, from 2019 until today?

In [21]:
qst9 = "SELECT strftime('%w', updated_at) AS weekly, ROUND( SUM(price_per_share * quantity ), 2) AS traded_value \
        FROM trades \
        GROUP BY weekly \
        ORDER BY traded_value DESC"
qst9_answer = cursor.execute(qst9).fetchall()

print(qst9_answer)

[('5', 518158.27), ('4', 11328.19), ('1', 1006.58), (None, 201.39), ('3', 53.95), ('2', 15.76)]


QUESTION 10: What are the top 5 bought stocks by volume? 

In [22]:
qst10 = "SELECT stock_symbol, ROUND( SUM(quantity ), 2) AS traded_volume \
         FROM trades \
         GROUP BY stock_symbol \
         ORDER BY traded_volume DESC"
qst10_answer = cursor.execute(qst10).fetchall()

print(qst10_answer)

[('TELL', 6472.28), ('EXPR', 3201.81), ('AMC', 2825.07), ('PHUN', 2769.82), ('NOK', 942.71), ('KOS', 687.97), ('GEVO', 496.89), ('SENS', 410.57), ('AHT', 372.3), ('ADXS', 345.04), ('SOLO', 335.64), ('GME', 282.91), ('BB', 252.02), ('INO', 245.34), ('GSAT', 223.15), ('ACRX', 161.84), ('DOFSQ', 149.18), ('ASNAQ', 147.75), ('MTCH', 139.0), ('M', 110.0), ('XPEV', 104.03), ('ASRT', 102.0), ('JMIA', 101.81), ('NIO', 100.53), ('ACOR', 98.16), ('IDEX', 88.02), ('YTRA', 87.93), ('SOL', 83.67), ('MARA', 81.76), ('BTU', 81.17), ('ERIC', 75.64), ('TSLA', 74.99), ('AQMS', 69.95), ('NKLA', 63.62), ('FCEL', 61.48), ('CCIV', 52.26), ('GNOG', 47.79), ('CHS', 44.61), ('SLB', 44.58), ('PLTR', 43.37), ('MGI', 43.3), ('CRIS', 42.88), ('FSR', 41.12), ('GORO', 39.85), ('CLOV', 37.17), ('WKHS', 36.55), ('WATT', 35.72), ('RUN', 32.14), ('ET', 29.84), ('AG', 28.35), ('AAPL', 27.41), ('FUBO', 26.66), ('NVAX', 26.61), ('TOUR', 26.32), ('DMYD', 25.86), ('VXRT', 22.67), ('LYG', 20.83), ('BLNK', 20.4), ('TNXP', 20.3

QUESTION 11: What are the trade commissions for each side? 

In [23]:
qst11 = "SELECT side, SUM(naira_fee) AS naira_commission, SUM(dollar_fee) AS dollar_commission \
         FROM trades GROUP BY side"
qst11_answer = cursor.execute(qst11).fetchall()

print(qst11_answer)
pd.read

[('BUY', 734657.4469999992, 1520.9399999999987), ('SELL', 2707372.2699999996, 5623.639999999999)]


QUESTION 12: Which stock generated the most commissions from the BUY side?

In [24]:
qst12 = "SELECT stock_symbol, SUM(naira_fee) AS naira_commission \
         FROM trades WHERE side = 'BUY' \
         GROUP BY stock_symbol \
         ORDER BY naira_commission DESC"
qst12_answer = cursor.execute(qst12).fetchall()

print(qst12_answer)

[('GME', 186739.84000000008), ('AMC', 71905.76000000002), ('TSLA', 51542.05), ('NVAX', 42814.75), ('GEVO', 26692.66), ('EXPR', 26145.96), ('XPEV', 24003.2), ('INO', 22771.06), ('PHUN', 20763.989999999998), ('AAPL', 19708.92), ('NOK', 19168.720000000005), ('MARA', 11541.470000000001), ('RUN', 11126.160000000002), ('BB', 10730.879999999997), ('PLTR', 8957.02), ('NIO', 8832.48), ('SOL', 8241.6), ('KOS', 7630.75), ('NKLA', 7181.7), ('ERIC', 6244.259999999999), ('SOLO', 5770.48), ('FUBO', 5331.6900000000005), ('NET', 4848.0), ('SLB', 4848.0), ('SHOP', 4513.49), ('GNOG', 4120.8), ('TELL', 3994.3700000000003), ('VOO', 3909.3), ('DMYD', 3636.0), ('GRWG', 3369.36), ('CCIV', 3342.41), ('NFLX', 3336.09), ('CRIS', 3317.8699999999994), ('FCEL', 2885.7400000000002), ('SENS', 2699.99), ('AG', 2665.38), ('AQMS', 2608.2200000000003), ('FSR', 2424.0), ('BTU', 2350.69), ('MRNA', 2339.6), ('FB', 2336.44), ('APPN', 2269.42), ('LMND', 2250.71), ('RIOT', 2230.08), ('LAZR', 2116.21), ('IDEX', 1928.95999999999

QUESTION 13: Count all users who deposited since bamboo began but haven't traded

In [25]:
qst13 = "SELECT COUNT (DISTINCT user_id) \
         FROM deposites \
         WHERE user_id NOT IN (SELECT user_id FROM trades)"
qst13_answer = cursor.execute(qst13).fetchall()

print(qst13_answer)

[(614,)]


QUESTION 14: Count all users with deposits under $1k

In [26]:
qst14 = "SELECT COUNT (DISTINCT user_id) \
        FROM deposites \
        WHERE amount_paid  < 1000"
qst14_answer = cursor.execute(qst14).fetchall()

print(qst14_answer)

[(94,)]


QUESTION 15: Count all users who had 2 or more deposits before 2021 but have not deposited in 2021

In [27]:
qst15 = "WITH new AS ( \
             SELECT user_id, COUNT ( user_id) AS users_count \
              FROM deposites \
              WHERE updated_at LIKE '2020%' \
              AND user_id NOT IN (SELECT user_id FROM deposites WHERE updated_at LIKE '2021%' ) \
              GROUP BY user_id \
              HAVING users_count >= 2) \
        SELECT count(user_id) \
        FROM new"
        
qst15_answer = cursor.execute(qst15).fetchall()

print(qst15_answer)

[(18,)]


In [28]:
con.close()